In [1]:
import logging
import os

for log_file in [ "logger.log", "error.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"Log file {log_file} are cleaned.")

logging.basicConfig(
    filename='logger.log',
    level=logging.DEBUG,
    format= '%(filename)s:%(lineno)s %(levelname)s: %(message)s'
)

print("Logging configured.")


Log file logger.log are cleaned.
Logging configured.


In [2]:
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.tools.agent_tool import AgentTool
from google.adk.tools.google_search_tool import google_search
from google.genai import types
from typing import List
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.agents.base_agent import BaseAgent
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.plugins.base_plugin import BasePlugin
from google.adk.plugins.logging_plugin import LoggingPlugin

In [3]:
retry_config = types.HttpRetryOptionsDict(
    attempts=5,
    exp_base= 7,
    initial_delay= 1,
    http_status_codes=[429,500,503,504]
)

In [4]:
def paper_count(papers: list[str]):
    """  This function counts the number of papers in a list of strings.
    Args:
      papers: A list of strings, where each string is a research paper.
    Returns:
      The number of papers in the list."""
    
    return len(papers)

In [5]:
google_search_agent = LlmAgent(
    name="google_search_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Use the google_search tool to find information on the given topic. Return the raw search results.
    If the user asks for a list of papers, then give them the list of research papers you found and not the summary.""",
    tools=[google_search]
)

In [6]:
root_agent = LlmAgent(
    name= "root_agent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options= retry_config),
    instruction="""Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_agent'. 
    2) Then, pass the papers to 'count_papers' tool to count the number of papers returned.
    3) Return both the list of research papers and the total number of papers.""",
    tools=[paper_count, AgentTool(agent=google_search_agent)]
)

In [7]:
async def create_session(name:str):

    app_name = name
    session_service = InMemorySessionService()
    session = await session_service.create_session(
        user_id="default", session_id= "Aamish", app_name= name
    )

    return session_service

In [ ]:
class countinvocationplugin(BasePlugin):
    """A custom Plugin that counts agent and tool invocations"""
    def __init__(self)-> None:
        """Initialize the plugin with counters."""
        super().__init__(name= "count_invocation")
        self.agent_count: int = 0
        self.tool_count: int = 0
        self.llm_request_count: int = 0


    async def before_agent_callback(self, *, agent: BaseAgent, callback_context: CallbackContext) -> None:
        """count agent run"""
        self.agent_count += 1
        logging.info(f"[plugin] agent invocations: {self.agent_count}")
        
    async def after_model_callback(self, *, callback_context:CallbackContext, llm_response: LlmRequest) -> None:
        self.llm_request_count += 1
        logging.info(f"[plugin] llm requests: {self.llm_request_count}")

In [12]:
APP_NAME = "agent"
start_session = await create_session(APP_NAME)
runner = Runner(agent = root_agent, app_name= APP_NAME, session_service= start_session, plugins=[LoggingPlugin()])

In [13]:
user_text = "what are the research on alopecia areta?"

# 1. Message Object tayar karein
message_object = types.Content(
    role="user",
    parts=[types.Part(text=user_text)]
)

async for event in runner.run_async(
    user_id='default',
    session_id= "Aamish",
    new_message=message_object
):
    if event.is_final_response():
        if event.content and event.content.parts:
            text = event.content.parts[0].text
            print("Final Response:", text)

[logging_plugin] 🚀 USER MESSAGE RECEIVED
[logging_plugin]    Invocation ID: e-21de088e-ca4b-4a4b-b7d9-36b61e0684cc
[logging_plugin]    Session ID: Aamish
[logging_plugin]    User ID: default
[logging_plugin]    App Name: agent
[logging_plugin]    Root Agent: root_agent
[logging_plugin]    User Content: text: 'what are the research on alopecia areta?'
[logging_plugin] 🏃 INVOCATION STARTING
[logging_plugin]    Invocation ID: e-21de088e-ca4b-4a4b-b7d9-36b61e0684cc
[logging_plugin]    Starting Agent: root_agent
[logging_plugin] 🤖 AGENT STARTING
[logging_plugin]    Agent Name: root_agent
[logging_plugin]    Invocation ID: e-21de088e-ca4b-4a4b-b7d9-36b61e0684cc
[logging_plugin] 🧠 LLM REQUEST
[logging_plugin]    Model: gemini-2.5-flash-lite
[logging_plugin]    Agent: root_agent
[logging_plugin]    System Instruction: 'Your task is to find research papers and count them. 

    You MUST ALWAYS follow these steps:
    1) Find research papers on the user provided topic using the 'google_search_ag